### ¿Qué es un Agente LLM?

Un **Agente LLM** es un sistema avanzado de inteligencia artificial que utiliza grandes modelos de lenguaje (como GPT-4, Llama, Claude, etc.) para comprender, generar y actuar sobre lenguaje humano de manera autónoma y contextual.

A diferencia de un simple `chatbot`, un agente LLM no solo responde preguntas, sino que puede planificar, razonar, recordar información previa, interactuar con herramientas externas (APIs, bases de datos, buscadores, etc.) y ejecutar tareas complejas de varios pasos, todo ello guiado por el lenguaje natural.

Una definición simple de un Agente:

> Un agente es un programa en el que el LLM controla el flujo de trabajo.

### Características principales de un Agente LLM

- **Comprensión profunda del lenguaje:** Interpreta intenciones, matices y contexto de las conversaciones humanas.
- **Generación de texto:** Produce respuestas coherentes y naturales, desde simples aclaraciones hasta textos complejos o resúmenes.
- **Memoria y contexto:** Puede recordar interacciones previas y mantener coherencia a lo largo de una conversación o flujo de trabajo.
- **Planificación y razonamiento:** Descompone problemas complejos en pasos más pequeños y decide qué acciones tomar y en qué orden.
- **Uso de herramientas externas:** Integra APIs, bases de datos, sistemas de búsqueda y otras utilidades para ampliar sus capacidades más allá del texto.
- **Autonomía:** Es capaz de ejecutar flujos de trabajo completos, tomar decisiones y adaptarse a los cambios en el entorno o en las instrucciones del usuario.



### Agentes de Dataframes Pandas

En esta práctica vamos a usar un agente LLM para trabajar con dataframes de Pandas. El agente LLM va a poder ejecutar código Python para manipular dataframes, y va a poder usar herramientas como `pandas`, `numpy`, `matplotlib`, etc. para realizar tareas complejas de análisis de datos.

Leemos las variables de entorno:

In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [2]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Introduce la APY Key de Gemini: ")
if "KAGGLE_USERNAME" not in os.environ:
    os.environ["KAGGLE_USERNAME"] = getpass.getpass("Introduce el usuario de Kaggle: ")
if "KAGGLE_KEY" not in os.environ:
    os.environ["KAGGLE_KEY"] = getpass.getpass("Introduce la API Key de Kaggle: ")


Leemos un `dataset` de Kaggle:

In [3]:
!kaggle datasets download -d goyaladi/climate-insights-dataset -p tmp  

Dataset URL: https://www.kaggle.com/datasets/goyaladi/climate-insights-dataset
License(s): CC0-1.0
climate-insights-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
import zipfile

# Define the path to your zip file
file_path = 'tmp/climate-insights-dataset.zip'
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall('datasets/climate_change')


Vamos a usar [este](https://www.kaggle.com/datasets/goyaladi/climate-insights-dataset) `dataset`:

In [ ]:
import pandas as pd
csv_file='datasets/climate_change/climate_change_data.csv'
document = pd.read_csv(csv_file)
document.head(5)

Creamos el agente:

In [17]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

sm_ds_Chat = create_pandas_dataframe_agent(
    ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=1),
    document,
    verbose=True,
    allow_dangerous_code=True, # LangChain obliga a poner esto. Ver: please see: https://python.langchain.com/docs/security/
)

Probamos el agente:

In [18]:
sm_ds_Chat.invoke("Analiza estos datos y haz un resumen de 100 palabras en español")



> Entering new AgentExecutor chain...
Thought: Para realizar un resumen de los datos, primero necesito entender las variables disponibles y sus posibles rangos. La tabla muestra información climática y ambiental recopilada en diferentes ubicaciones y países a lo largo del tiempo. Incluye la fecha, la ubicación, el país, la temperatura, las emisiones de CO2, el aumento del nivel del mar, las precipitaciones, la humedad y la velocidad del viento. Para hacer un buen resumen, calcularé estadísticas descriptivas para cada columna.

Action: python_repl_ast
Action Input: ```python
import pandas as pd
import numpy as np

summary = df.describe()
print(summary)
```        Temperature  CO2 Emissions  Sea Level Rise  Precipitation  \
count  10000.000000   10000.000000    10000.000000   10000.000000   
mean      14.936034     400.220469       -0.003152      49.881208   
std        5.030616      49.696933        0.991349      28.862417   
min       -3.803589     182.131220       -4.092155       0.

{'input': 'Analiza estos datos y haz un resumen de 100 palabras en español',
 'output': 'El conjunto de datos presenta información climática y ambiental recopilada de diversas ubicaciones alrededor del mundo. Abarca un período de tiempo con registros de temperatura, emisiones de CO2, aumento del nivel del mar, precipitación, humedad y velocidad del viento. La temperatura promedio es de alrededor de 14.9 grados, con una variación considerable. Las emisiones de CO2 promedian 400 unidades, mientras que el nivel del mar muestra una media cercana a cero, indicando fluctuaciones. La precipitación también varía ampliamente, al igual que la humedad. La velocidad del viento presenta una media de 25 unidades. Los datos provienen de diversos países y ubicaciones, con algunas regiones representadas con mayor frecuencia que otras.'}

Podemos hacer gráficos. Observe que el agente falla y descubre que la columna que recoge la temperatura tiene un espacio al final ("Temperature "). Lo tiene en cuenta y vuelve a ejecutar el gráfico:

In [19]:
sm_ds_Chat.invoke("Dibuja la relación entre la temperatura y la humedad para todo el dataset")



> Entering new AgentExecutor chain...
Thought: The question asks to draw the relationship between temperature and humidity, which means generating a scatter plot with temperature on one axis and humidity on the other.
Action: [python_repl_ast]
Action Input: `import matplotlib.pyplot as plt
plt.scatter(df['Temperature'], df['Humidity'])
plt.xlabel('Temperature')
plt.ylabel('Humidity')
plt.title('Relationship between Temperature and Humidity')
plt.show()`[python_repl_ast] is not a valid tool, try one of [python_repl_ast].The question asks to draw the relationship between temperature and humidity, which means generating a scatter plot with temperature on one axis and humidity on the other.

Action: [python_repl_ast]
Action Input: ```python
import matplotlib.pyplot as plt
plt.scatter(df['Temperature'], df['Humidity'])
plt.xlabel('Temperature')
plt.ylabel('Humidity')
plt.title('Relationship between Temperature and Humidity')
plt.show()
```[python_repl_ast] is not a valid tool, try one of 

{'input': 'Dibuja la relación entre la temperatura y la humedad para todo el dataset',
 'output': 'The final answer is the python code above.'}

Los campos que puede usar un agente son:
Estudiando el rastro (trace), se pueden observar tres campos principales:

- **Thought (Pensamiento):** Muestra el razonamiento interno del agente, indicando lo que planea hacer y cuál es su objetivo inmediato. Es como su diálogo interno, donde analiza la situación y decide el siguiente paso a seguir.

- **Action (Acción):** Refleja las acciones que realiza el agente, normalmente llamando a funciones de Python o herramientas externas a las que tiene acceso, como búsquedas en internet, consultas a bases de datos, cálculos, etc

- **Observation (Observación):** Es la información o los datos que obtiene como resultado de la acción realizada. El agente utiliza esta observación para ajustar su razonamiento y decidir su próximo objetivo o acción, repitiendo el ciclo hasta alcanzar la meta final.
  
Este ciclo de Pensamiento → Acción → Observación permite a los agentes LLM resolver tareas complejas de manera iterativa, integrando razonamiento, ejecución y aprendizaje en tiempo real.


Otra pregunta:

In [21]:
sm_ds_Chat.invoke("¿Crees que se puede predecir la temperatura?, ¿Cómo?")



> Entering new AgentExecutor chain...
Thought: Yes, I believe temperature can be predicted using the other columns in the dataframe as features. I would use a machine learning model, like a regression model, to achieve this.

Final Answer: Sí, creo que la temperatura se puede predecir. Se puede utilizar un modelo de aprendizaje automático de regresión, usando las otras columnas del dataframe como características para predecir la temperatura.


> Finished chain.


{'input': '¿Crees que se puede predecir la temperatura?, ¿Cómo?',
 'output': 'Sí, creo que la temperatura se puede predecir. Se puede utilizar un modelo de aprendizaje automático de regresión, usando las otras columnas del dataframe como características para predecir la temperatura.'}

Otra más:

In [22]:
sm_ds_Chat.invoke("""Selecciona un modelo estadístico de predicción para pronosticar la temperatura.
Utiliza este tipo de modelo para predecir la temperatura media
anual en South David, Vietnam, para los próximos 5 años.
Escribe las temperaturas pronosticadas en una tabla.""")




> Entering new AgentExecutor chain...
Thought: First, I need to calculate the average annual temperature for South David, Vietnam. Then, I'll use a simple linear regression model to forecast the temperature for the next 5 years, assuming a linear trend. Finally, I'll present the forecasted temperatures in a table.

Action: python_repl_ast
Action Input: ```python
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Assuming df is already loaded as in the problem description

# Filter data for South David, Vietnam
df['Date'] = pd.to_datetime(df['Date'])
south_david_vietnam = df[(df['Location'] == 'South David') & (df['Country'] == 'Vietnam')]

# Extract year and temperature
south_david_vietnam['Year'] = south_david_vietnam['Date'].dt.year
annual_avg_temp = south_david_vietnam.groupby('Year')['Temperature'].mean().reset_index()

# Linear Regression for forecasting
model = LinearRegression()
model.fit(annual_avg_temp[['Year']], annual_avg_temp['Temp

<string>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/surtich/projects/Ampliaci-n-de-Entornos/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


   Year  Predicted Temperature
0  2001              12.309581
1  2002              12.309581
2  2003              12.309581
3  2004              12.309581
4  2005              12.309581
Final Answer:

|   Year |   Predicted Temperature |
|-------:|------------------------:|
|   2001 |                 12.3096 |
|   2002 |                 12.3096 |
|   2003 |                 12.3096 |
|   2004 |                 12.3096 |
|   2005 |                 12.3096 |


> Finished chain.


{'input': 'Selecciona un modelo estadístico de predicción para pronosticar la temperatura.\nUtiliza este tipo de modelo para predecir la temperatura media\nanual en South David, Vietnam, para los próximos 5 años.\nEscribe las temperaturas pronosticadas en una tabla.',
 'output': '|   Year |   Predicted Temperature |\n|-------:|------------------------:|\n|   2001 |                 12.3096 |\n|   2002 |                 12.3096 |\n|   2003 |                 12.3096 |\n|   2004 |                 12.3096 |\n|   2005 |                 12.3096 |'}